In [1]:
import pandas as pd

In [5]:
ozone_df = pd.read_csv("오존 2015-2025.csv")
so2_df = pd.read_csv("아황산가스 2015-2025.csv")
no2_df = pd.read_csv("이산화질소 2015-2025.csv")
co_df = pd.read_csv("일산화탄소 2015-2025.csv")

In [6]:
ozone_long = ozone_df.melt(id_vars=["날짜"], var_name="region", value_name="ozone")
so2_long = so2_df.melt(id_vars=["날짜"], var_name="region", value_name="so2")
no2_long = no2_df.melt(id_vars=["날짜"], var_name="region", value_name="no2")
co_long = co_df.melt(id_vars=["날짜"], var_name="region", value_name="co")

In [9]:
import datetime

def fix_24_hour_format(dt_str):
    if '24:00' in dt_str:
        date_part = dt_str.split()[0]
        new_dt = pd.to_datetime(date_part) + pd.Timedelta(days=1)
        return f"{new_dt.strftime('%Y-%m-%d')} 00:00"
    return dt_str

for df in [ozone_long, so2_long, no2_long, co_long]:
    df.rename(columns={"날짜": "datetime"}, inplace=True)
    df["datetime"] = df["datetime"].apply(fix_24_hour_format)
    df["datetime"] = pd.to_datetime(df["datetime"])


In [10]:
merged = pd.merge(ozone_long, so2_long, on=["datetime", "region"], how="inner")
merged = pd.merge(merged, no2_long, on=["datetime", "region"], how="inner")
merged = pd.merge(merged, co_long, on=["datetime", "region"], how="inner")

In [11]:
merged

,datetime,region,ozone,so2,no2,co
0,2015-01-01 01:00:00,평균,0.0210,0.0054,0.0099,0.36
1,2015-01-01 02:00:00,평균,0.0201,0.0055,0.0096,0.37
2,2015-01-01 03:00:00,평균,0.0197,0.0054,0.0090,0.38
3,2015-01-01 04:00:00,평균,0.0203,0.0053,0.0074,0.35
4,2015-01-01 05:00:00,평균,0.0195,0.0052,0.0074,0.34
...,...,...,...,...,...,...
2350993,2025-06-25 13:00:00,중랑구,0.0271,0.0024,0.0097,0.24
2350994,2025-06-25 14:00:00,중랑구,0.0290,0.0023,0.0081,0.24
2350995,2025-06-25 15:00:00,중랑구,0.0255,0.0029,0.0108,0.26
2350996,2025-06-25 16:00:00,중랑구,0.0237,0.0022,0.0099,0.26


In [13]:
merged.rename(columns={"ozone": "o3"}, inplace=True)

In [14]:
merged

,datetime,region,o3,so2,no2,co
0,2015-01-01 01:00:00,평균,0.0210,0.0054,0.0099,0.36
1,2015-01-01 02:00:00,평균,0.0201,0.0055,0.0096,0.37
2,2015-01-01 03:00:00,평균,0.0197,0.0054,0.0090,0.38
3,2015-01-01 04:00:00,평균,0.0203,0.0053,0.0074,0.35
4,2015-01-01 05:00:00,평균,0.0195,0.0052,0.0074,0.34
...,...,...,...,...,...,...
2350993,2025-06-25 13:00:00,중랑구,0.0271,0.0024,0.0097,0.24
2350994,2025-06-25 14:00:00,중랑구,0.0290,0.0023,0.0081,0.24
2350995,2025-06-25 15:00:00,중랑구,0.0255,0.0029,0.0108,0.26
2350996,2025-06-25 16:00:00,중랑구,0.0237,0.0022,0.0099,0.26


In [15]:
merged['region'].unique()

array(['평균', '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구',
       '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구',
       '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'],
      dtype=object)

In [16]:
import json
from sqlalchemy import create_engine 
from sqlalchemy.exc import SQLAlchemyError
import cryptography

In [17]:
with open('db-config.json', 'r') as f:
    config = json.load(f)

user = config['user']
password = config['password']
host = config['host']
port = config['port']
database = config['database']

merged = merged.copy() 
merged['datetime'] = pd.to_datetime(merged['datetime'])  # datetime 형식으로 변환

In [19]:
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')
conn = engine.connect()

try:
    merged.to_sql(name='air_pollution', con=conn, if_exists='append', index=False)
    print('데이터 삽입 완료')

except SQLAlchemyError as e:
    print('오류 발생', str(e))

데이터 삽입 완료


In [20]:
import pandas as pd
from sqlalchemy import create_engine
import json

# 접속 정보 로딩 (옵션: db-config.json 파일이 있을 경우)
with open('db-config.json') as f:
    config = json.load(f)

user = config['user']
password = config['password']
host = config['host']
port = config['port']
database = config['database']

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# vehicle_reg_stats 테이블 데이터 읽어오기
query = "SELECT * FROM air_pollution"
df = pd.read_sql(query, con=engine)

# ▶ 결과 확인
print(df.head())

      id            datetime region     no2    co     so2      o3
0  14907 2015-01-01 01:00:00     평균  0.0099  0.36  0.0054  0.0210
1  14908 2015-01-01 02:00:00     평균  0.0096  0.37  0.0055  0.0201
2  14909 2015-01-01 03:00:00     평균  0.0090  0.38  0.0054  0.0197
3  14910 2015-01-01 04:00:00     평균  0.0074  0.35  0.0053  0.0203
4  14911 2015-01-01 05:00:00     평균  0.0074  0.34  0.0052  0.0195


In [21]:
df['region'].unique()

array(['평균', '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구',
       '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구',
       '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'],
      dtype=object)